# Pytorch Lightning 적응기

- Task : KLUE-DP
- 담당자: [권지현](https://github.com/Jihyun22) 님
- 최종수정일: 21-09-08
- 본 자료는 가짜연구소 3기 KLUE 로 모델 평가하기 크루 활동으로 작성됨

# 01 init
공식 KLUE github 의 [KLUE-baseline](https://github.com/KLUE-benchmark/KLUE-baseline) 을 바탕으로 작성된 노트북입니다. 공식 KLUE github 을 다룰 때 필요한 기초적인 환경 설정 방법에 대해 안내합니다.

---

## Install packages

우선 KLUE 벤치마크 데이터와 Fine-tuning 을 위한 Pipeline 을 가져오겠습니다. 공식 KLUE github 을 clone 하여 관련 자료를 사용할 수 있습니다.

In [ ]:
!git clone --recursive https://github.com/KLUE-benchmark/KLUE-Baseline.git

Cloning into 'KLUE-Baseline'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 67 (delta 13), reused 63 (delta 9), pack-reused 0
Unpacking objects: 100% (67/67), done.
Submodule 'data' (https://github.com/KLUE-benchmark/KLUE) registered for path 'data'
Cloning into '/content/KLUE-Baseline/data'...
remote: Enumerating objects: 154, done.        
remote: Counting objects: 100% (154/154), done.        
remote: Compressing objects: 100% (134/134), done.        
remote: Total 154 (delta 42), reused 42 (delta 4), pack-reused 0        
Receiving objects: 100% (154/154), 41.65 MiB | 14.98 MiB/s, done.
Resolving deltas: 100% (42/42), done.
Submodule path 'data': checked out '1cc52e64c0e0b6915577244f7439c55a42199a64'


공식 Github 에 명시된 필요 패키지도 설치합니다.

In [ ]:
!pip install -r KLUE-Baseline/requirements.txt

Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     |████████████████████████████████| 776.7 MB 4.7 kB/s 
     |████████████████████████████████| 1.3 MB 40.8 MB/s 
     |████████████████████████████████| 665 kB 52.1 MB/s 
     |████████████████████████████████| 108 kB 54.8 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 1.1 MB 39.4 MB/s 
     |████████████████████████████████| 895 kB 42.5 MB/s 
     |████████████████████████████████| 2.9 MB 14.3 MB/s 
     |████████████████████████████████| 829 kB 39.0 MB/s 
     |████████████████████████████████| 636 kB 43.8 MB/s 
  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10186 sha256=6f24e4cd0caba9d916e2baa724b7db821da958237f995de0b4214581f7751bc0
  Stored in directory: /root/.cache/pip/wheels/3a/0d/38/01a9bc6e20dcfaf0a6a7b552d03137558ba1c38aea47644682
  Created wheel 

이때, colab 환경에서 실행할 경우 requirements 에 명시된 패키지 중 버전을 고려하여 추가 설치가 필요한 패키지가 있습니다(`torchtext`, `torch`, `pytorch-lightning`). 해당 패키지도 추가로 설치합니다(21-08-25 기준).

In [ ]:
!pip install pytorch-lightning==1.2.2 torchtext==0.8.0 torch==1.7.1

     |████████████████████████████████| 816 kB 4.2 MB/s 
     |████████████████████████████████| 6.9 MB 17.2 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 269 kB 48.9 MB/s 
     |████████████████████████████████| 1.3 MB 36.1 MB/s 
     |████████████████████████████████| 142 kB 50.4 MB/s 
     |████████████████████████████████| 294 kB 51.7 MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44635 sha256=510cae55419c17018ff4f38b70fc6ea0d3c101934f264838884d8ab82c6a0182
  Stored in directory: /root/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
Successfully built PyYAML
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
 

환경 세팅이 완료되었습니다.

---

## 01 Train

공식 Github 에 Fine-tuning 을 위한 pipeline 이 명시되어 있습니다. 본 노트북에서는 해당 pipeline 을 활용하여 학습을 수행하겠습니다.


Task 별 개별 학습을 위해서는 `run_klue.py` 을 사용합니다. Fine-tuning 을 위한 인자 목록은 다음과 같습니다(활용 시 편의를 위해 정리해 두었으므로 자세한 내용은 공식 자료를 참조바랍니다).

Pytorch Lightning 의 [자료](https://pytorch-lightning.readthedocs.io/en/latest/advanced/training_tricks.html)를 참고하였으며, 기본값은 `run_klue.py` 에 명시된 값을 따릅니다.

- --task : 학습을 수행할 task 입니다. 총 8개의 task 가 있습니다.
  - 입력 예시 : ['ynat', 'klue-nli', 'klue-sts', 'klue-ner', 'klue-re', 'klue-dp', 'klue-mrc', 'wos']
- --output_dir : 학습 결과(model checkpoint 등)가 저장될 path 입니다.
- --gpus : GPU 를 사용하여 학습할 경우 할당할 GPU number 입니다. colab 에서 gpu 런타임으로 사용할 경우 0으로 입력합니다.
- --fp16 : 16 bit precision 을 사용할 경우 설정합니다.
  - 입력 예시 : ['true','false']
  - 기본값은 `false` 입니다.
- --num_sanity_val_steps : validation steps 의 수 입니다.
  - 기본값은 2입니다.
- --n_tpu_cores : TPU 를 사용하여 학습할 경우 할당할 TPU number 입니다.
- --max_grad_norm : Gradient clipping 을 설정하는 경우 norm 값을 설정합니다.
  - 기본값은 1 입니다.
- --accumulate_grad_batches : Accumulate gradients 학습 시 배치 크기를 설정할 수 있습니다.
  - 기본값은 1 입니다.
- --seed : 재현성을 위한 random seed를 설정합니다.
  - 기본값은 42 입니다.
- --metric_key : 사용할 evaluation metrics 을 설정합니다.
  - 입력 예시 : ['train/loss_arc', 'train/loss_type', 'train/loss', 'valid/uas_macro_f1', 'valid/uas_micro_f1', 'valid/las_macro_f1', 'valid/las_micro_f1'] 
- --patience : early_stopping_mode 에 따라 학습에 개선이 없을 경우 종료할 epoch 수 입니다.
  - 기본값은 5 입니다.
- --early_stopping_mode : training 과정을 모니터링 하며 학습이 개선되지 않을 경우 종료하는 mode 입니다. 학습 평가 시 evaluation metrics 을 고려하여 `min`, `max` 중 선택하여 입력할 수 있습니다.
  - 입력 예시 : ['min', 'max'] 
  - 기본값은 `max` 입니다.
- --data_dir : 데이터의 위치 입니다. 
- --train_file_name  :
- --dev_file_name :
- --test_file_name : 
- --num_workers :
  - 기본값은 4 입니다.
- --train_batch_size 
- --eval_batch_size 
- --max_seq_length 
- --model_name_or_path 
- --config_name 
- --tokenizer_name 
- --cache_dir 
- --encoder_layerdrop 
- --decoder_layerdrop 
- --dropout 
- --attention_dropout 
- --learning_rate 
- --lr_scheduler 
- --weight_decay 
- --adam_epsilon 
- --warmup_steps 
- --warmup_ratio 
- --max_epochs 
- --adafactor 
- --verbose_step_count 
- --encoder_layers 
- --decoder_layers 
- --hidden_size 
- --arc_space 
- --type_space 
- --no_pos 
- --pos_dim 


DP task 의 평가지표는 uas_micro_f1, las_micro_f1 입니다. 우선 las_micro_f1에 대한 학습을 수행하겠습니다. 사용 모델은 `klue/roberta-base` 입니다.

In [ ]:
!python KLUE-Baseline/run_klue.py \
train \
--task 'klue-dp' \
--output_dir ./dp \
--metric_key 'las_micro_f1' \
--data_dir KLUE-Baseline/data/klue_benchmark/klue-dp-v1.1/ \
--model_name_or_path klue/roberta-base \
--learning_rate 3e-5 \
--num_train_epochs 5 \
--train_batch_size 64 \
--eval_batch_size 16 \
--max_seq_length 510 \
--max_grad_norm 0.5 \
--warmup_ratio 0.1 \
--weight_decay 0.01 \
--gpus 0 \

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: MetricBase will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
08/25/2021 11:38:37 - INFO - __main__ - Arguments:
08/25/2021 11:38:37 - INFO - __main__ - 	                command : train
08/25/2021 11:38:37 - INFO - __main__ - 	                   task : klue-dp
08/25/2021 11:38:37 - INFO - __main__ - 	             output_dir : ./dp
08/25/2021 11:38:37 - INFO - __main__ - 	                   gpus : [0]
08/25/2021 11:38:37 - INFO - __main__ - 	                   fp16 : False
08/25/2021 11:38:37 - INFO - __main__ - 	   num_sanity_val_steps : 2
08/25/2021 11:38:37 - INFO - __main__ - 	              tpu_cores : None
08/25/2021 11:38:37 - INFO - __main__ - 	      gradient_clip_val : 0.5
08/25/2021 11:38:37 - INFO - __main__ - 	accumulate_grad_batches : 1
08/25/2021 11:38:37 - INFO - __main__ - 	           

# 02 Test

- 추가할 예정

# Reference
- https://arxiv.org/pdf/2105.09680.pdf